### Mission RHOBS


Description des colonnes :

Colonne A : Identifiant de chaque personne (ID).

Colonne B : Description détaillée de chaque personne.

Colonne C : Prénom de chaque personne.

Colonne D : Nom de famille de chaque personne.

Colonne E : Richesse totale de chaque personne.

Colonne F : Possible dette, indiquant une diminution de la richesse.

Colonne G : Richesse nette.

Ce code lit un fichier CSV en chunks, extrait les valeurs numériques de la colonne 'e' et les associe à des noms et prénoms de personnes à partir des colonnes 'c' et 'd'. Ensuite, il identifie les trois plus grandes valeurs de la colonne 'e' et imprime les noms et prénoms correspondants avec ces valeurs.

Voici ce que fait chaque partie du code :

1. La fonction `process_chunk` est chargée du traitement de chaque chunk de lignes du fichier CSV. Pour chaque ligne du chunk :
   - Elle divise la ligne en colonnes en utilisant `strip().split(',')`.
   - Elle vérifie si la ligne a au moins 5 colonnes et si la colonne 7 (index 6 en Python) est numérique.
   - Elle extrait le nom et le prénom des personnes à partir des colonnes 'c' et 'd'.
   - Elle construit une clé à partir des colonnes 'c' et 'd' et associe la valeur numérique de la colonne 'e' à cette clé dans le dictionnaire `key_values_pairs`. Si la clé n'existe pas ou si la nouvelle valeur est plus grande que la valeur existante, elle met à jour le dictionnaire.
   
2. La fonction `read_csv_chunks` lit le fichier CSV en chunks de taille spécifiée.
   - Elle ouvre le fichier CSV en mode lectu next(file)`.
   - Elle lit le fichier en chunks à l'aide de `islice` et les traite en parallèle à l'aide de `ThreadPoolExecutor`. Chaque chunk est traité par un thread distinct.
   - Après avoir traité tous les chunks, elle récupère les trois plus grandes valeurs de la colonne 'e' avec les noms et prénoms correspondants et les imprime.

In [5]:
import concurrent.futures
from itertools import islice

def process_chunk(chunk, key_values_pairs):
    for line in chunk:
        columns = line.strip().split(',')
        if len(columns) >= 5 and columns[6].replace('.', '', 1).isdigit():
            key_s = columns[2] + columns[3]
            valueur = int(columns[6])
            name = columns[2]  # Nom de la personne
            surname = columns[3]  # Prénom de la personne
            if key_s not in key_values_pairs or key_values_pairs[key_s][0] < valueur:
                key_values_pairs[key_s] = (valueur, name, surname)

def read_csv_chunks(file_path, chunk_size):
    key_values_pairs = {}
    with open(file_path, 'r') as file:
        next(file)  # Ignorer la première ligne
        while True:
            chunk = list(islice(file, chunk_size))
            if not chunk:
                break
            with concurrent.futures.ThreadPoolExecutor() as executor:
                executor.map(lambda c: process_chunk(c, key_values_pairs), [chunk])

    # Récupérer les trois plus grandes valeurs et leurs noms
    sorted_values = sorted(key_values_pairs.values(), key=lambda x: x[0], reverse=True)[:3]
    print("The three greatest values are:")
    for value, name, surname in sorted_values:
        print(f"{name} {surname}: {value}")

file_path = "test.csv"
chunk_size = 10000

read_csv_chunks(file_path, chunk_size)


The three greatest values are:
ZoÃ© Walliand: 893775958
Gerard Parmentier: 892220563
OcÃ©ane Urbain: 891910460
